In [ ]:
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def set_random_seed(seed):
    torch.manual_seed(seed)  # PyTorch

    # Ensures that PyTorch gets the same random numbers for CUDA as well, if using GPU.
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False




def attention(P,Q,Z, activation = None):
    B= Z.shape[0]
    N = Z.shape[1]-1
    d = Z.shape[2]-1
    A = torch.eye(N+1).to(device)
    A[N,N] = 0
    Attn = torch.einsum('BNi, ij, BMj -> BNM', (Z,Q,Z))
    if activation is not None:
        Attn = activation(Attn)
    key = torch.einsum('ij, BNj -> BNi', (P,Z))
    Output = torch.einsum('BNM,ML, BLi -> BNi', (Attn,A,key))
    return Output /N



class Transformer_MLP(nn.Module):
    def __init__(self, n_layer, n_head, d, var, hidden_dim):
        super(Transformer_MLP, self).__init__()
        self.register_parameter('allparam', torch.nn.Parameter(torch.zeros(n_layer, n_head, 2, d+1, d+1)))
        with torch.no_grad():
            self.allparam.normal_(0,var)
        self.n_layer = n_layer
        self.n_head = n_head
        self.mlp = nn.Sequential(nn.Linear(d, hidden_dim),nn.ReLU(),nn.Linear(hidden_dim, d))

    def forward(self, Z):
        Z[:,:,:-1] = self.mlp(Z[:,:,:-1].clone())
        for i in range(self.n_layer):
            Zi = Z
            residues = 0
            # the forwarad map of each layer is given by F(Z) = Z + attention(Z)
            for j in range(self.n_head):
                Pij = self.allparam[i,j,0,:,:]
                Qij = self.allparam[i,j,1,:,:]
                residues = residues + attention(Pij,Qij,Zi)
            Z = Zi + residues
        return Z

# evaluate the loss of model, given data (Z,y)
def in_context_loss(model, Z, y):
    N = Z.shape[1]-1
    d = Z.shape[2]-1
    output = model(Z)
    diff = output[:,N,d]+y
    loss = ((diff)**2).mean()
    return loss

def generate_data_mlp(N, B, d, randomMLP):
    # Generate random input data
    X = torch.FloatTensor(B, N, d).normal_(0, 1).to(device)
    X_test = torch.FloatTensor(B, 1, d).normal_(0, 1).to(device)

    # Additional transformations if mode is 'sphere' or 'gamma' [Similar to the existing generate_data function]

    X_MLP = randomMLP(X.view(-1, d)).view(B, N, d)
    X_test_MLP = randomMLP(X_test.view(-1, d)).view(B, 1, d)

    W = torch.FloatTensor(B, d).normal_(0,1).cuda()
    y = torch.einsum('bi,bni->bn', (W, X_MLP)).unsqueeze(2)
    y_zero = torch.zeros(B,1,1).cuda()
    y_test = torch.einsum('bi,bni->bn', (W, X_test_MLP)).squeeze(1)
    X_comb= torch.cat([X,X_test],dim=1)
    y_comb= torch.cat([y,y_zero],dim=1)
    Z= torch.cat([X_comb,y_comb],dim=2)

    return Z, y_test
# Setup
N=20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_layer = 3  # number of layers of transformer
d = 5        # dimension for the data
n_head = 1   # 1-headed attention
B = 2000     # minibatch size for sine data
var = 0.0001 # initializations scale of transformer parameter
clip =1


tuning_epochs = 400
max_iters = 2000  # Number of Iterations to run
hidden_dim = 10
learning_rates = [0.001, 0.01, 0.05, 0.1]

# Function to run training
def train_transformer(optimizer_name, learning_rate, max_epochs):
    randomMLP = nn.Sequential(nn.Linear(d, hidden_dim),nn.ReLU(),nn.Linear(hidden_dim, d)).to(device)
    for para in randomMLP.parameters():
        para.requires_grad = False
    # Initialize model and optimizer

    # Instantiate model
    model = Transformer_MLP(n_layer, n_head, d, var, 3* hidden_dim).to(device)

    # Wrap the model for DataParallel
    if torch.cuda.device_count() > 1:
        print(f"Let's use {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)

    if optimizer_name == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))

    losses = []
    for epoch in range(max_epochs):
        # Generate data
        Z, y = generate_data_mlp(N=N, B=B, d=d, randomMLP=randomMLP)

        # Move data to the correct device
        Z, y = Z.to(device), y.to(device)

        # Training step
        optimizer.zero_grad()
        loss = in_context_loss(model, Z, y)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()

        optimizer.zero_grad()

        # Record loss
        losses.append(loss.item())

        # Optionally print progress
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    return losses

# Tuning learning rate

optimal_lr = {}
for optimizer_name in ['adam','sgd']:
    best_lr = None
    best_loss = float('inf')
    for lr in learning_rates:
        print(f"Training with {optimizer_name} optimizer, Learning Rate: {lr}")
        losses = train_transformer(optimizer_name, lr, tuning_epochs)
        avg_loss = np.mean(losses[-20:])  # Average loss over last epochs
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_lr = lr
    optimal_lr[optimizer_name] = best_lr
    print(f"Optimal Learning Rate for {optimizer_name}: {best_lr}")

num_runs = 5
final_losses = {}
for optimizer_name, lr in optimal_lr.items():
    print(f"Extended Training with {optimizer_name} optimizer, Learning Rate: {lr}")
    all_losses = []
    for seed in range(num_runs):
        set_random_seed(seed)  # Function to set the random seed, define it as needed
        losses = train_transformer(optimizer_name, lr, max_iters)
        all_losses.append(losses)

    # Calculate the mean and standard deviation of the losses
    mean_losses = np.mean(all_losses, axis=0)
    std_losses = np.std(all_losses, axis=0)
    final_losses[optimizer_name] = (mean_losses, std_losses)

# Plotting losses with shaded error bar
plt.figure(figsize=(10, 6))
epochs = range(len(mean_losses))  # Assuming all runs have the same number of epochs
for optimizer_name, (mean_losses, std_losses) in final_losses.items():
    plt.plot(epochs, mean_losses, label=f'{optimizer_name} LR={optimal_lr[optimizer_name]}')
    plt.fill_between(epochs, mean_losses - std_losses, mean_losses + std_losses, alpha=0.3)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.legend()
plt.show()

Training with adam optimizer, Learning Rate: 0.001
Epoch 0, Loss: 0.37667232751846313
Epoch 100, Loss: 0.16827353835105896
Epoch 200, Loss: 0.1396411508321762
Epoch 300, Loss: 0.13879340887069702
Training with adam optimizer, Learning Rate: 0.01
Epoch 0, Loss: 0.2456808239221573
Epoch 100, Loss: 0.052687838673591614
Epoch 200, Loss: 0.027845948934555054
Epoch 300, Loss: 0.013793528079986572
Training with adam optimizer, Learning Rate: 0.05
Epoch 0, Loss: 0.26487094163894653
Epoch 100, Loss: 0.03570482134819031
Epoch 200, Loss: 0.028097547590732574
Epoch 300, Loss: 0.008483467623591423
Training with adam optimizer, Learning Rate: 0.1
Epoch 0, Loss: 0.5550039410591125
Epoch 100, Loss: 0.19777850806713104
Epoch 200, Loss: 0.1297670602798462
Epoch 300, Loss: 0.11323829740285873
Optimal Learning Rate for adam: 0.01
Training with sgd optimizer, Learning Rate: 0.001
Epoch 0, Loss: 0.9390906095504761
Epoch 100, Loss: 1.0138301849365234
Epoch 200, Loss: 0.9242967963218689
Epoch 300, Loss: 1.000